# Data Import

## Import libraries

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
%matplotlib inline

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## Import datasets

The datasets we use here include: 
    - IMDB:
        - Data source: 
            - https://www.imdb.com/interfaces/
        - Dataset: 
            - imdb.title.ratings
            - imdb.title.basics
        - Documentation:
            - These datasets were filtered to contain only movies from 2010 to 2018
    - The-Numbers.com
        - Data source: 
            - https://www.the-numbers.com/movie/budgets/all)
        - Dataset:
            - tn.movie_budgets
        - Documentation: 
            - This includes all data from The Numbers

In [2]:
imdb_title_ratings = pd.read_csv('data/imdb.title.ratings.csv')
imdb_title_basics = pd.read_csv('data/imdb.title.basics.csv')
tn_movie_budgets = pd.read_csv('data/tn.movie_budgets.csv')

## Merge datasets and rename columns

In [3]:
imdb_joined = pd.merge(imdb_title_basics, imdb_title_ratings, on='tconst', how='left')

In [4]:
imdb_joined['title'] = imdb_joined['primary_title']

In [5]:
imdb_joined['year'] = imdb_joined['start_year']

In [6]:
tn_movie_budgets['title'] = tn_movie_budgets['movie']

In [7]:
tn_movie_budgets['year'] = tn_movie_budgets['release_date'].map(lambda x:x[-4:]).astype(int)

In [8]:
df = pd.merge(tn_movie_budgets, imdb_joined, on =["title", "year"], how ='left')

# Data Cleaning

Change data types

In [9]:
df['production_budget'] = df['production_budget'].map(lambda x:float(x.replace("$", "").replace(",", "")))
df['domestic_gross'] = df['domestic_gross'].map(lambda x:float(x.replace("$", "").replace(",", "")))
df['worldwide_gross'] = df['worldwide_gross'].map(lambda x:float(x.replace("$", "").replace(",", "")))

In [10]:
df['release_date'] = pd.to_datetime(df['release_date'])
df['release_year_month'] = df['release_date'].dt.to_period("M")

Drop duplicated movies in the column of df.title

In [11]:
df = df.drop_duplicates(['title'])

Drop unnecessary columns

In [12]:
df.drop(['id', 'movie', 'tconst', 'primary_title', 'original_title', 'start_year'], axis = 1, inplace = True)

Check and drop for missing values

In [13]:
df.isna().sum()

release_date             0
production_budget        0
domestic_gross           0
worldwide_gross          0
title                    0
year                     0
runtime_minutes       4246
genres                4230
averagerating         4252
numvotes              4252
release_year_month       0
dtype: int64

In [14]:
df = df[~df['runtime_minutes'].isna()]
df = df[~df['genres'].isna()]
df = df[~df['averagerating'].isna()]
df = df[~df['numvotes'].isna()]

In [15]:
df.isna().sum()

release_date          0
production_budget     0
domestic_gross        0
worldwide_gross       0
title                 0
year                  0
runtime_minutes       0
genres                0
averagerating         0
numvotes              0
release_year_month    0
dtype: int64

Drop itemss in the columns of df['domestic_gross'] and df['worldwide_gross'] that have values equal to 0

In [16]:
df.drop(df.index[df['domestic_gross'] == 0], inplace = True)
df.drop(df.index[df['worldwide_gross'] == 0], inplace = True)

Create two new variables df['domestic_profit'] and df['worldwide_profit']

In [17]:
df['domestic_profit'] = df['domestic_gross'] - df['production_budget']
df['worldwide_profit'] = df['worldwide_gross'] - df['production_budget']

Rearrange columns

In [21]:
df = df[['title', 'genres', 'runtime_minutes',\
        'year', 'release_year_month', 'release_date', 'averagerating',\
        'numvotes', 'production_budget', 'domestic_gross', 'domestic_profit',\
         'worldwide_gross', 'worldwide_profit']]